In [1]:
import sys
import numpy as np

sys.path.append('/home/dylan/NEXUS_RF/BackendTools')
import MB_equations as MBe

## Device Parameters
Define and calculate some numbers that are needed for later calculations. The $\alpha$ and $\Delta$ values were measured for a sister device at Caltech. 

In [2]:
## Define the MB parameters
alpha = 0.03801
Delta = 0.184         ## meV
D     = Delta*1e-3    ##  eV

## Temperature of the device
T = 10.0e-3 ## K

## Superconducting volume per resonator
V = 3e4 ## um^3

## Energy of the incident photons
hv = 2.61 ## eV (475 nm)

## Readout settings
rf_power = -72.5   ## dBm at USRP
f        = 4.241985e9 / 1e6   ## MHz

In [3]:
k1 = MBe.kappa_1(T, f*1e6, D) *1e6*1e6*1e6
k2 = MBe.kappa_2(T, f*1e6, D) *1e6*1e6*1e6

print("Readout Power:", rf_power, "dBm")
print("Readout Freq.:", f , "MHz")
print("Device Temp.: ", T*1e3, "mK")
print("==============================")
print("alpha: ", alpha*100, "%")
print("Delta: ", Delta    , "meV")
print("V:     ", V        , "um^3")
print("k1:    ", k1       , "um^3")
print("k2:    ", k2       , "um^3")

Readout Power: -72.5 dBm
Readout Freq.: 4241.985 MHz
Device Temp.:  10.0 mK
alpha:  3.801 %
Delta:  0.184 meV
V:      30000.0 um^3
k1:     2.276385718161626e-07 um^3
k2:     3.9131623591500685e-07 um^3


## Resolution from Noise
We can determine an estimate for the resolution in each readout unit by running an OF on noise data. In this case, we use the average pulse from the largest LED voltage data as the signal template, and run it on the second half (pulse-free region) of the lowest LED voltage data.

In [4]:
## These numbers came from July Data (max=6.0 V)
resolutions_from_noise = { ## [max pulse height , norm baseline res , readout res]
    "df" : [6.96625263820683e-07, 0.007779148914891695, 5.419151665136807e-09],
    "dQ" : [4.346751025013649e-07, 0.024587726973304063, 1.0687672742396518e-08],
    "dk1": [203.68695317546957, 0.024587726973304063, 5.008199192702615],
    "dk2": [18.499778323924932, 0.007779148914891696, 0.14391253047429756]
}

In [5]:
## These numbers came from February Data
resolutions_from_noise = { ## [max pulse height , norm baseline res , readout res]
    "df" : [4.652203226819528e-07, 0.007788494922923435, 3.6233661212491918e-09],
    "dQ" : [2.2831555009892842e-07, 0.03239491433878988, 7.396262687668476e-09],
    "dk1": [106.98715901911099, 0.03239491433878989, 3.4658398517745925],
    "dk2": [12.354523764471535, 0.0077884949229234356, 0.09622314561472349]
}

In [6]:
## Print the numbers that came from the OF in each unit
print("df/f:", resolutions_from_noise["df"][2])
print("d1/Q:", resolutions_from_noise["dQ"][2])
print("dk_2:", resolutions_from_noise["dk2"][2])
print("dk_1:", resolutions_from_noise["dk1"][2])

df/f: 3.6233661212491918e-09
d1/Q: 7.396262687668476e-09
dk_2: 0.09622314561472349
dk_1: 3.4658398517745925


The fractional frequency shift and excess quasiparticle density are related by the equation $$\frac{\delta f}{f} = - \frac{\alpha}{2} \kappa_2 n_\mathrm{qp} $$ which implies the relation between resolution in $\delta f/f$ is related to the resolution in quasiparticle density by $$ \sigma_{df/f} = \frac{\alpha}{2} \kappa_2 \sigma_{n_\mathrm{qp}} $$ or $$ \sigma_{n_\mathrm{qp}} = \frac{2}{\alpha \kappa_2} \sigma_{df/f} $$ as is implemented in the `quasiparticle_basis` conversion.

In [7]:
sigma_k2_calc = 2./(k2 * alpha) * resolutions_from_noise["df"][2]
print("dk_2 resolution, calculated:", sigma_k2_calc, "um^-3")
print("this is a factor", sigma_k2_calc/resolutions_from_noise["dk2"][2], "larger than found by OF")

dk_2 resolution, calculated: 0.4872103190319991 um^-3
this is a factor 5.06333809728882 larger than found by OF


Likewise, we can calculate the resolution in quasiparticle fluctuations in the $\kappa_1$ direction using the equation $$ \delta \frac{1}{Q} = \alpha \kappa_1 n_\mathrm{qp} $$ and as such $$ \sigma_{n_\mathrm{qp}} = \frac{1}{\alpha \kappa_1} \sigma_{1/Q} $$

In [8]:
sigma_k1_calc = 1./(k1 * alpha) * resolutions_from_noise["dQ"][2]
print("dk_1 resolution, calculated:", sigma_k1_calc, "um^-3")
print("this is a factor", resolutions_from_noise["dk1"][2]/sigma_k1_calc, "smaller than found by OF")

dk_1 resolution, calculated: 0.8548080540486652 um^-3
this is a factor 4.054524095040029 smaller than found by OF


Now we can compute the resolution on energy absorbed by the superconductor, by using the volume of the superconductor and the energy it takes to break a Cooper pair: $$ \sigma_E^\mathrm{abs} = \sigma_{n_\mathrm{qp}} V \Delta $$

In [9]:
sig_E_abs_eV = sigma_k2_calc * V * D
print("Resolution on energy absorbed by superconductor:",sig_E_abs_eV,"eV")

Resolution on energy absorbed by superconductor: 2.689400961056635 eV


## Resolution from Pulse Amplitude Histogram Fits

In [10]:
## This is the fit result for r from the July(?) data
s0_from_fit = -1
r_from_fit  = -1

In [11]:
## This is the fit result for r from the February data (df/f)
s0_from_fit = 0.007548731381363116
r_from_fit  = 6.187766079809401e-05

From the fit value of $\sigma_0$ and the maximum frational frequency shift pulse height $s_{df/f}$, we can determine the resolution on qp density: $$ \sigma_\mathrm{qp}^{\kappa_2} = \frac{2}{\alpha \kappa_2} (\sigma_0 s_{df/f} ) $$

In [12]:
sig_qp_k2_fit = (2/(alpha*k2)) * s0_from_fit * resolutions_from_noise['df'][0]
print("QP resolution from fit:",sig_qp_k2_fit,"um^-3")

QP resolution from fit: 0.47221187931651193 um^-3


And from that we calculate the resolution on energy absorbed by the superconductor in the usual way $$ \sigma_E^\mathrm{abs} = \sigma_\mathrm{qp} V \Delta $$

In [13]:
sig_E_abs_eV_fit = sig_qp_k2_fit * V * D
print("Resolution on energy absorbed by superconductor:",sig_E_abs_eV_fit,"eV (from fit)")

Resolution on energy absorbed by superconductor: 2.6066095738271455 eV (from fit)


Since we fit to an equation of the form $$ \sigma_E = \sqrt{\sigma_0^2 + r \times \mu} $$ where $r$ is the responsivity per photon, we can determine the responsivity per unit energy by dividing by the photon energy $$ R \equiv r/(h\nu) $$

In [14]:
R = r_from_fit/hv
print("Responsivity per eV:",R)

Responsivity per eV: 2.3707916014595405e-05


The resolution in our responsivity can be converted to resolution on energy deposited in the substrate via $$ \sigma_E^\mathrm{dep} = \frac{ \sigma_0 }{R} $$ where $\sigma_0$ is in our normalized units.

In [15]:
sig_E_dep_eV_fit = s0_from_fit / R
print("Resolution on energy deposited in substrate:",sig_E_dep_eV_fit,"eV")

Resolution on energy deposited in substrate: 318.40552230385236 eV


From the canonical definition of $\eta_\mathrm{ph}$, we can find $$ \eta_\mathrm{ph} = \frac{\sigma_E^\mathrm{abs}}{\sigma_E^\mathrm{dep}} $$

In [16]:
eta_ph = sig_E_abs_eV_fit / sig_E_dep_eV_fit
print("Energy collection efficiency:", eta_ph*100, "%")

Energy collection efficiency: 0.8186445872442107 %
